In [1]:
# !wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
# !dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
# !apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
# !apt update -q
# !apt install cuda gcc-6 g++-6 -y -q
# !ln -s /usr/bin/gcc-6 /usr/local/cuda/bin/gcc
# !ln -s /usr/bin/g++-6 /usr/local/cuda/bin/g++

In [2]:
# !curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.7/julia-1.7.3-linux-x86_64.tar.gz" -o julia.tar.gz
# !tar -xzf julia.tar.gz -C /usr --strip-components 1
# !rm -rf julia.tar.gz*
# !julia -e 'using Pkg; pkg"add IJulia; precompile"'

# OLS and lasso for wage prediction

## Introduction

In labor economics an important question is what determines the wage of workers. This is a causal question, but we could begin to investigate from a predictive perspective.

In the following wage example, $Y$ is the hourly wage of a worker and $X$ is a vector of worker's characteristics, e.g., education, experience, gender.
Two main questions here are:    

* How to use job-relevant characteristics, such as education and experience, to best predict wages?

* What is the difference in predicted wages between men and women with the same job-relevant characteristics?

In this lab, we focus on the prediction question first.

## Data

The data set we consider is from the March Supplement of the U.S. Current Population Survey, year 2015. We select white non-hispanic individuals, aged 25 to 64 years, and working more than 35 hours per week during at least 50 weeks of the year. We exclude self-employed workers; individuals living in group quarters; individuals in the military, agricultural or private household sectors; individuals with inconsistent reports on earnings and employment status; individuals with allocated or missing information in any of the variables used in the analysis; and individuals with hourly wage below $3$.

The variable of interest $Y$ is the hourly wage rate constructed as the ratio of the annual earnings to the total number of hours worked, which is constructed in turn as the product of number of weeks worked and the usual number of hours worked per week. In our analysis, we also focus on single (never married) workers. The final sample is of size $n = 5150$.

## Data Analysis

In [3]:
# to_install = ["CSV", "DataFrames", "Dates", "Plots"]
# using Pkg 
# Pkg.add(to_install)
# PKG.add("Lathe")
# Pkg.add("HTTP")
using CSV, DataFrames, Dates, Plots, Lathe, GLM, Statistics, MLBase, HTTP

We start by loading the data set.

In [4]:
#Reading the CSV file into a DataFrame
#We have to set the category type for some variable
url = "https://github.com/d2cml-ai/14.388_jl/raw/main/data/wage2015_subsample_inference.csv"
data = CSV.File(download(url); types = Dict("occ" => String,"occ2"=> String,"ind"=>String,"ind2"=>String)) |> DataFrame
size(data)

(5150, 21)

Let's look at the structure.

In [5]:
#a quick decribe of the data
describe(data)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,rownames,15636.3,10,15260.0,32643,0,Int64
2,wage,23.4104,3.02198,19.2308,528.846,0,Float64
3,lwage,2.97079,1.10591,2.95651,6.2707,0,Float64
4,sex,0.444466,0.0,0.0,1.0,0,Float64
5,shs,0.023301,0.0,0.0,1.0,0,Float64
6,hsg,0.243883,0.0,0.0,1.0,0,Float64
7,scl,0.278058,0.0,0.0,1.0,0,Float64
8,clg,0.31767,0.0,0.0,1.0,0,Float64
9,ad,0.137087,0.0,0.0,1.0,0,Float64


We are constructing the output variable $Y$ and the matrix $Z$ which includes the characteristics of workers that are given in the data.

In [6]:
n = size(data)[1]
z = select(data, Not([:rownames, :lwage, :wage]))
p = size(z)[2] 

println("Number of observations : ", n, "\n","Number of raw regressors: ", p )

Number of observations : 5150
Number of raw regressors: 18


For the outcome variable wage and a subset of the raw regressors, we calculate the empirical mean to get familiar with the data.

In [7]:

z_subset = select(data, ["lwage","sex","shs","hsg","scl","clg","ad","mw","so","we","ne","exp1"])
rename!(z_subset, ["Log Wage", "Sex", "Some High School", "High School Graduate", "Some College", "College Graduate", "Advanced Degree", "Midwest", "South", "West", "Northeast", "Experience"])

describe(z_subset, :mean)

,variable,mean
,Symbol,Float64
1,Log Wage,2.97079
2,Sex,0.444466
3,Some High School,0.023301
4,High School Graduate,0.243883
5,Some College,0.278058
6,College Graduate,0.31767
7,Advanced Degree,0.137087
8,Midwest,0.259612
9,South,0.296505


E.g., the share of female workers in our example is ~44% ($sex = 1$ if female).

## Prediction Question


Now, we will construct a prediction rule for hourly wage $Y$ , which depends linearly on job-relevant characteristics  $X$:

$$Y = \beta' X + \epsilon $$
 
Our goals are

* Predict wages using various characteristics of workers.

* Assess the predictive performance using the (adjusted) sample MSE, the (adjusted) sample $R^2$ and the out-of-sample $MSE$ and $R^2$.

We employ two different specifications for prediction:

- **Basic Model**: $X$ consists of a set of raw regressors (e.g. gender, experience, education indicators, occupation and industry indicators, regional indicators).

- **Flexible Model**: $X$ consists of all raw regressors from the basic model plus occupation and industry indicators, transformations (e.g.,$exp2$ and $exp3$) and additional two-way interactions of polynomial in experience with other regressors. An example of a regressor created through a two-way interaction is experience times the indicator of having a college degree.

Using the **Flexible Model**, enables us to approximate the real relationship by a more complex regression model and therefore to reduce the bias. The **Flexible Model** increases the range of potential shapes of the estimated regression function. In general, flexible models often deliver good prediction accuracy but give models which are harder to interpret.

Now, let us fit both models to our data by running ordinary least squares (ols):

In [8]:
#basic model
basic  = @formula(lwage ~ (sex + exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2))
basic_results  = lm(basic, data)
println(basic_results)
println("Number of regressors in the basic model: ", size(coef(basic_results), 1))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{

Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}



lwage ~ 1 + sex + exp1 + shs + hsg + scl + clg + mw + so + we + occ2 + ind2

Coefficients:


─────────────────────────────────────────────────────────────────────────────────
                   Coef.   Std. Error       t  Pr(>|t|)    Lower 95%    Upper 95%
─────────────────────────────────────────────────────────────────────────────────


(Intercept)   3.52838     0.0540195    

 65.32    <1e-99   3.42248      3.63429
sex          -0.0728575   0.0150269     -4.85    <1e-05  -0.102317    -0.0433983
exp1          0.0085677   0.000653744   13.11    <1e-37   0.00728608   0.00984932
shs          -0.592798    0.0505549    -11.73    <1e-30  -0.691908    -0.493689
hsg          -0.504337    0.0270767    -18.63    <1e-74  -0.557419    -0.451255
scl          -0.411994    0.0252036    -16.35    <1e-57  -0.461404    -0.362584
clg          -0.182216    0.0229524     -7.94    <1e-14  -0.227213    -0.137219
mw           -0.0275413   0.0193301     -1.42    0.1543  -0.0654367    0.010354
so           -0.0344538   0.0187063     -1.84    0.0656  -0.0711262    0.00221853
we            0.0172492   0.020086       0.86    0.3905  -0.022128     0.0566264
occ2: 10     -0.0106235   0.0396274     -0.27    0.7886  -0.0883102    0.0670633
occ2: 11     -0.455834    0.0594409     -7.67    <1e-13  -0.572364    -0.339305
occ2: 12     -0.307589    0.0555146     -5.54    <1e-07  -0.416421    -0.

In [9]:
#flexible model
flex = @formula(lwage ~ sex + shs+hsg+scl+clg+occ2+ind2+mw+so+we + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we))
flex_results = lm(flex, data)
println(flex_results)
println("Number of regressors in the flexible model: ", size(coef(flex_results), 1))

##### Re-estimating the flexible model using lasso

We re-estimate the flexible model using Lasso (the least absolute shrinkage and selection operator) rather than ols. Lasso is a penalized regression method that can be used to reduce the complexity of a regression model when the ratio $p/n$ is not small. We will introduce this approach formally later in the course, but for now, we try it out here as a black-box method.

In [25]:
using Lasso

lasso_model = fit(LassoModel, flex, data)

StatsModels.TableRegressionModel{LassoModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, MinAICc}, Matrix{Float64}}

lwage ~ sex + shs + hsg + scl + clg + occ2 + ind2 + mw + so + we + exp1 + exp2 + exp3 + exp4 + exp1 & shs + exp1 & hsg + exp1 & scl + exp1 & clg + exp1 & occ2 + exp1 & ind2 + exp1 & mw + exp1 & so + exp1 & we + exp2 & shs + exp2 & hsg + exp2 & scl + exp2 & clg + exp2 & occ2 + exp2 & ind2 + exp2 & mw + exp2 & so + exp2 & we + exp3 & shs + exp3 & hsg + exp3 & scl + exp3 & clg + exp3 & occ2 + exp3 & ind2 + exp3 & mw + exp3 & so + exp3 & we + exp4 & shs + exp4 & hsg + exp4 & scl + exp4 & clg + exp4 & occ2 + exp4 & ind2 + exp4 & mw + exp4 & so + exp4 & we

Coefficients:
LassoModel using MinAICc(2) segment of the regularization path.

Coefficients:
──────────────────
          Estimate
──────────────────
x1     3.34129
x2    -0.0631407
x3    -0.565639
x4    -0.50156
x5    -0.400049
x6    -0.14691

##### Evaluating the predictive performance of the basic and flexible models
Now, we can evaluate the performance of both models based on the (adjusted) $R^2_{sample}$ and the (adjusted) $MSE_{sample}$:

In [42]:
n_data = size(data)[1]

function ref_bsc(model, lasso = false, n = n_data)
     if lasso
        p = length(coef(model))
        y_hat = predict(model)
        y_r = data.lwage
        r_2 = 1 - sum((y_r .- y_hat).^2)  / sum((y_r .- mean(y_r)).^2)
        adj_r2 = 1 - (1 - r_2) * ((n - 1) / (n - p - 1))
    else
        p = length(coef(model))
        r_2 = r2(model)
        adj_r2 = adjr2(model)
    end   
    
    mse = mean(residuals(model).^2)
    mse_adj = (n / (n - p)) * mse
    
    ref = [p, r_2, adj_r2, mse, mse_adj]
    
    return p, r_2, adj_r2, mse, mse_adj
end

p1, r2_1, r2_adj1, mse1, mse_adj1 = ref_bsc(basic_results);

p2, r2_2, r2_adj2, mse2, mse_adj2 = ref_bsc(flex_results);

pL, r2_L, r2_adjL, mseL, mse_adjL = ref_bsc(lasso_model, true);

println("R-squared for the basic model: ", r2_1)
println("Adjusted R-squared for the basic model: ", r2_adj1)
println("R-squared for the flexible model: ", r2_2)
println("Adjusted R-squared for the flexible model: ", r2_adj2)
println("R-squared for the lasso with flexible model: ", r2_2)
println("Adjusted R-squared for the lasso with flexible model: ", r2_adj2, "\n")

println("MSE for the basic model: ", mse1)
println("MSE for the basic model: ", mse_adj1)
println("MSE for the flexible model: ", mse2)
println("MSE for the flexible model: ", mse_adj2)
println("MSE for the lasso with flexible model: ", mseL)
println("MSE for the lasso with flexible model: ", mse_adjL)


R-squared for the basic model: 0.31004650692219493
Adjusted R-squared for the basic model: 0.303280930406429
R-squared for the flexible model: 0.35110989506172274
Adjusted R-squared for the flexible model: 0.3186918535221881
R-squared for the lasso with flexible model: 0.35110989506172274
Adjusted R-squared for the lasso with flexible model: 0.3186918535221881

MSE for the basic model: 0.22442505581164396
MSE for the basic model: 0.2266697465051905
MSE for the flexible model: 0.2110681364431821
MSE for the flexible model: 0.22165597526149833
MSE for the lasso with flexible model: 0.21912180704256773
MSE for the lasso with flexible model: 0.23011364320334907


In [9]:
# using Pkg
# Pkg.add("Lasso")
using Lasso

flex = @formula(lwage ~ sex + shs+hsg+scl+clg+occ2+ind2+mw+so+we + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we))
lasso_model = fit(LassoModel, flex, data)

StatsModels.TableRegressionModel{LassoModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, MinAICc}, Matrix{Float64}}

lwage ~ sex + shs + hsg + scl + clg + occ2 + ind2 + mw + so + we + exp1 + exp2 + exp3 + exp4 + exp1 & shs + exp1 & hsg + exp1 & scl + exp1 & clg + exp1 & occ2 + exp1 & ind2 + exp1 & mw + exp1 & so + exp1 & we + exp2 & shs + exp2 & hsg + exp2 & scl + exp2 & clg + exp2 & occ2 + exp2 & ind2 + exp2 & mw + exp2 & so + exp2 & we + exp3 & shs + exp3 & hsg + exp3 & scl + exp3 & clg + exp3 & occ2 + exp3 & ind2 + exp3 & mw + exp3 & so + exp3 & we + exp4 & shs + exp4 & hsg + exp4 & scl + exp4 & clg + exp4 & occ2 + exp4 & ind2 + exp4 & mw + exp4 & so + exp4 & we

Coefficients:
LassoModel using MinAICc(2) segment of the regularization path.

Coefficients:
──────────────────
          Estimate
──────────────────
x1     3.34129
x2    -0.0631407
x3    -0.565639
x4    -0.50156
x5    -0.400049
x6    -0.14691

In [38]:
# lasso_model, basic_results, regflex
n_data = size(data)[1]

function ref_bsc(model, lasso = false, n = n_data)
     if lasso
        p = length(coef(model))
        y_hat = predict(model)
        y_r = data.lwage
        r_2 = 1 - sum((y_r .- y_hat).^2)  / sum((y_r .- mean(y_r)).^2)
        adj_r2 = 1 - (1 - r_2) * ((n - 1) / (n - p - 1))
    else
        p = length(coef(model))
        r_2 = r2(model)
        adj_r2 = adjr2(model)
    end   
    
    mse = mean(residuals(model).^2)
    mse_adj = (n / (n - p)) * mse
    
    ref = [p, r_2, adj_r2, mse, mse_adj]
    
    return ref
    
end

DataFrame(
    Model = ["p", "R^2", "MSE", "R^2 adjusted", "MSE adjusted"],
    Basic_reg = ref_bsc(basic_results),
    Flexible_reg = ref_bsc(flex_results),
    lasso_flex = ref_bsc(lasso_model, true)
)

,Model,Basic_reg,Flexible_reg,lasso_flex
,String,Float64,Float64,Float64
1,p,51.0,246.0,246.0
2,R^2,0.310047,0.35111,0.32635
3,MSE,0.303281,0.318692,0.292551
4,R^2 adjusted,0.224425,0.211068,0.219122
5,MSE adjusted,0.22667,0.221656,0.230114


Considering the measures above, the flexible model performs slightly better than the basic model. 

As $p/n$ is not large, the discrepancy between the adjusted and unadjusted measures is not large. However, if it were, we might still like to apply **data splitting** as a more general procedure to deal with potential overfitting if $p/n$. We illustrate the approach in the following.

## Data Splitting

Measure the prediction quality of the two models via data splitting:

- Randomly split the data into one training sample and one testing sample. Here we just use a simple method (stratified splitting is a more sophisticated version of splitting that we might consider).
- Use the training sample to estimate the parameters of the Basic Model and the Flexible Model.
- Use the testing sample for evaluation. Predict the $\mathtt{wage}$  of every observation in the testing sample based on the estimated parameters in the training sample.
- Calculate the Mean Squared Prediction Error $MSE_{test}$ based on the testing sample for both prediction models. 

In [44]:
using Lathe.preprocess: TrainTestSplit

In [45]:
train, test = TrainTestSplit(data, 4/5)
reg_basic = lm(basic, train)

train_reg_basic = predict(reg_basic, test)
y_test = test.lwage

mse_test1 = sum((y_test .- train_reg_basic).^2) / length(y_test)
r2_test1 = 1 - mse_test1 / var(y_test)

print("Test MSE for the basic model: $mse_test1\nTest R2 for the basic model: $r2_test1")

Test MSE for the basic model: 0.2185049608897876
Test R2 for the basic model: 0.32969404519304046

In the basic model, the $MSE_{test}$ jis quite close to the $MSE_{sample}$

In [13]:
reg_flex = lm(flex, train)
train_reg_flex = predict(reg_flex, test)
mse_test2 = sum((y_test .- train_reg_flex).^2) / length(y_test)
r2_test2 = 1 - mse_test2 / var(y_test)
    
print("Test MSE for the basic model: $mse_test2\nTest R2 for the basic model: $r2_test2")

Test MSE for the basic model: 0.2549752677075409
Test R2 for the basic model: 0.20895231863861619

In the flexible model too, the discrepancy between the $MSE_{test}$ and the $MSE_{sample}$ is not large.

It is worth noticing that the $MSE_{test}$ varies across different data splits. Hence, it is a good idea to average the out-of-sample MSE over different data splits to get valid results.

Nevertheless, we observe that, based on the out-of-sample $MSE$, the basic model using ols regression performs **about as well (or slightly better)** than the flexible model. 

Next, let us use lasso regression in the flexible model instead of ols regression. The out-of-sample $MSE$ on the test sample can be computed for any black-box prediction method, so we also compare the performance of lasso regression in the flexible model to ols regression.

In [14]:
reg_lasso = fit(LassoModel, flex, train)
train_reg_lasso = predict(reg_lasso, test)
mse_lasso = sum((y_test .- train_reg_lasso).^2) / length(y_test)
r2_lasso = 1 - mse_lasso / var(y_test)
print("Test MSE for the basic model: $mse_lasso\nTest R2 for the basic model: $r2_lasso")

Test MSE for the basic model: 0.22909929950928218
Test R2 for the basic model: 0.28923118187993957

Finally, let us summarize the results:

In [15]:
MSE = [mse_test1, mse_test2, mse_lasso]
R2 = [r2_test1, r2_test2, r2_lasso]
Model = ["Basic reg", "Flexible reg", "Lasso Regression"]
DataFrame( Model = Model, MSE_test = MSE, R2_test = R2)

,Model,MSE_test,R2_test
,String,Float64,Float64
1,Basic reg,0.229097,0.289239
2,Flexible reg,0.254975,0.208952
3,Lasso Regression,0.229099,0.289231
